# Batch Run Example

This example demonstrates how to run build cot Search Results on a batch of question/answer pairs using the CoTBuilder class. Results are saved to a files in jsonl format as well as saved in memory.

**Imports**

This example will use LMStudio for local model serving and OpenAI for generation via external API. CoT-Forge also supports major LLM providers like Gemini, Groq, and Anthropic. Download LMStudio [here](https://lmstudio.ai/), download a model and run a local server.

In [63]:
from datasets import load_dataset

from cot_forge import CoTBuilder, LLMJudgeVerifier, NaiveLinearSearch
from cot_forge.llm import AnthropicProvider, OpenAIProvider

#### Dataset
The dataset we will use is [medical-o1-verifiable-problem by FreedomIntelligence on HuggingFace](https://huggingface.co/datasets/FreedomIntelligence/medical-o1-verifiable-problem). This dataset contains a set of open-ended medical questions and ground truth answers.



In [64]:
ds = load_dataset("FreedomIntelligence/medical-o1-verifiable-problem", split="train[:200]")
problems = ds["Open-ended Verifiable Question"]
solutions = ds["Ground-True Answer"]

# Print the first problem and solution to see the data format
print(f"Problem #1: {problems[0]}")
print(f"Solution #1: {solutions[0]}")

Problem #1: An 88-year-old woman with osteoarthritis is experiencing mild epigastric discomfort and has vomited material resembling coffee grounds multiple times. Considering her use of naproxen, what is the most likely cause of her gastrointestinal blood loss?
Solution #1: Gastric ulcer


#### Using CoTBuilder class to run batch build

In [65]:
# We'l use meta-llama-3-8b-instruct as our search_llm.
# gpt-4o will be used for the verifier.
gpt_4o = OpenAIProvider(model_name="gpt-4o")
sonnet_3_5 = AnthropicProvider(model_name="claude-3-5-sonnet-20241022")

builder = CoTBuilder(
    search_llm=gpt_4o, # generates reasoning steps
    search=NaiveLinearSearch(), # Naive linear search chooses random reasoning steps in a chain
    verifier=LLMJudgeVerifier(llm_provider=sonnet_3_5, strict=False), # claude sonnet to verify answers
    post_processing_llm=sonnet_3_5, # converts reasoning into natural language
    dataset_name="medical-o1-verifiable-problem", # dataset name, used for folder structure
    base_dir= "./data", # base directory to save the results
)

Let's process our dataset using the process_batch() method of CoTBuilder. This method can be run with multi-threading to speed up the process or with a single thread. Because we supplied a dataset_name parameter, the results will be saved to a file in jsonl format.

In [66]:
results = builder.process_batch(
    questions=problems, # List of questions
    ground_truth_answers=solutions, # List of ground truth answers
    multi_thread=True, # Use multi-threading for processing
    max_workers=4, # Number of workers to use for processing
    load_processed=True, # Load previously processed results if available
    only_successful=False, # Only process successful results into natural language
    overwrite=True, # Overwrite existing results if any collisions occur
    limit=20 # Limit the number of questions to process
)

Multi-thread processing question and ground truth answer pairs.: 100%|██████████| 20/20 [02:58<00:00,  8.91s/pair]


#### Examining results

Results are returned as a list of tuples, each with a [SearchResult](../src/cot_forge/reasoning/types.py#L137) object and a dictionary with the natural language reasoning text.

In [96]:
sample_result, reasoning = results[12]
print(sample_result)

SearchResult(success=True, question=A 20-year-old female patient p..., num_terminal_nodes=1, num_successful_nodes=1, successful_answers=["Given the patient's age, MRI characteristics, and symptom of 6th cranial nerve palsy, the most probable diagnosis is a schwannoma, likely a trigeminal schwannoma affecting the cavernous sinus."])


Our search result was a success! Let's dig in further.

In [97]:
# Each SearchResult object contains a list of terminal nodes.
# Because we used a linear search, there is only one chain and therefore one terminal node.
terminal_node = sample_result.terminal_nodes[0]
for i, node in enumerate(terminal_node.get_full_node_chain()):
  print(f"Step {i}: {node}")

Step 0: ReasoningNode(strategy=initialize, success=False, final=False, cot_steps=6)
Step 1: ReasoningNode(strategy=validation, success=True, final=True, cot_steps=6)


### Natural language reasoning
Let's examine the natural language reasoning text from our sample result. Reasoning happens between the <reasoning> tags, followed by the final answer.

In [98]:
print(reasoning['chain_of_thought_responses'][0])

<thinking>Let me think about this case... A young woman with 6th cranial nerve palsy. That's going to affect her lateral rectus muscle, so she's probably experiencing double vision.

Okay, looking at the MRI findings - we've got a hyperintense lesion in the cavernous sinus that enhances homogeneously with contrast. Hmm, that's interesting.

What typically causes lesions in that area? Well, the usual suspects would be meningiomas, schwannomas, maybe cavernous sinus thrombosis... The homogeneous enhancement is pretty characteristic of meningiomas, actually.

Wait a minute - let me consider the patient's age. She's only 20 years old. While meningiomas do tend to occur more in females, they're usually seen in older patients. That makes me question my initial thinking.

Let me think about schwannomas for a moment... They also show up as hyperintense on T2 and enhance homogeneously. And trigeminal schwannomas, in particular, love to hang out in the cavernous sinus region.

Oh, and the age ac

#### Results storage

In our CoTBuilder object, we specified `dataset_name="medical-o1-verifiable-problem"` and `base_dir= "./data"`. This means that the results will be saved to a folder called `./data/medical-o1-verifiable-problem` in the current working directory. Additionally, another folder at `./data/medical-o1-verifiable-problem/naive_linear_search` was created to store the results of the naive linear search. If a different search algorithm were used, the folder name would reflect that and results would be stored there instead to avoid overwriting.

Generally, the results will be saved in a folder structure like this:

```bash
base_dir/
└── dataset_name/
    ├── search_algorithm/
    │   ├── config.json
    │   ├── metadata.json
    │   ├── results.jsonl
    │   └── reasoning.jsonl
```

In [99]:
import os

print(os.listdir("./data/medical-o1-verifiable-problem/naive_linear_search"))

['reasoning.jsonl', 'config.json', 'metadata.json', 'results.jsonl']


The files contain the following information:
- `config.json`: The configuration of the CoTBuilder object used to run the search.
- `metadata.json`: Metadata about search progress such as completed items, successful items, last updated time, etc.
- `results.jsonl`: Serialized [SearchResult](../src/cot_forge/reasoning/types.py#L137) objects in jsonl format. They can be deserialized using the SearchResult.deserialize() method.
- `reasoning.jsonl`: Dictionaries with the natural language reasoning text. Each dictionary contains the following keys:
  - `question`: The question that was asked.
  - `ground_truth`: The answer that was given.
  - `chain_of_thought_responses`: A list of strings, representing the processed reasoning text for each stored chain in a processed question.